In [19]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import base64

# Load your cleaned dataset
df = pd.read_csv('cleaned_dataset.csv')  # Replace with the path to your dataset

# Prepare a list of dictionaries for the dropdown options
crime_types = [{'label': str(crime_type), 'value': str(crime_type)} for crime_type in df['Primary Type'].unique()]

# Initialize the Dash app with callback exceptions suppressed
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

# Encode the image for use in Dash layout
image_filename = 'skyline.jpg'  # replace with your own image path
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

# Define the app layout
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        children=[
            dbc.NavItem(dbc.NavLink("Home", href="/")),
            dbc.NavItem(dbc.NavLink("Crime Types", href="/crime-types")),
            dbc.NavItem(dbc.NavLink("Locations", href="/locations")),
        ],
        brand="Chicago Crime Dashboard",
        brand_href="/",
        color="primary",
        dark=True,
    ),
    # Main content area where visualization components will be added
    html.Div(id='page-content')
])

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/crime-types':
        # Content for the 'Crime Types' page
        return html.Div([
            dcc.Dropdown(
                id='crime-type-dropdown',
                options=crime_types,
                value=crime_types[0]['value'],  # Default value
                clearable=False
            ),
            dcc.Graph(id='crime-type-line-chart')
        ])
    elif pathname == '/locations':
        # Placeholder for location visualization components
        return html.Div([
            dcc.Graph(id='crime-location-heatmap'),
            # Add more components as necessary
        ])
    else:
        # Homepage content
        return html.Div([
            # Background image with text overlay
            html.Div([
                html.Div([
                    html.H1("Welcome to the Chicago Crime Dashboard", className="text-overlay"),
                    html.P("Explore interactive visualizations to understand crime trends in Chicago.", className="text-overlay")
                ], className="image-text-container"),
            ], style={
                'backgroundImage': f'url(data:image/jpg;base64,{encoded_image})',
                'backgroundSize': 'cover',
                'backgroundPosition': 'center',
                'height': '95vh',  # Adjust the height as needed
                'position': 'relative',
                'color': 'white'
            }),
            # Other homepage content
        ], style={'textAlign': 'center', 'color': 'white'})

# CSS styles for the text overlay
app.css.append_css({
    "external_url": [
        "https://codepen.io/chriddyp/pen/bWLwgP.css",  # This is just an example; include your own CSS file as needed
        # Add your own external CSS file here
    ]
})

# Callback for updating the line chart based on the selected crime type
@app.callback(
    Output('crime-type-line-chart', 'figure'),
    [Input('crime-type-dropdown', 'value')]
)
def update_line_chart(selected_crime_type):
    # Filter the DataFrame based on the selected crime type
    filtered_df = df[df['Primary Type'] == selected_crime_type]

    # Group by year and count the occurrences
    yearly_counts = filtered_df.groupby('Year').size().reset_index(name='Count')

    # Create the line chart figure
    fig = px.line(yearly_counts, x='Year', y='Count', title=f'Total {selected_crime_type} Crimes per Year')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


/opt/anaconda3/lib/python3.9/site-packages/dash/resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of ['https://codepen.io/chriddyp/pen/bWLwgP.css'] is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

